# Plant-Disease Relation using BiLSTM-CRF Model approach

In [ ]:
! pip install pandas==0.23.4
! pip install keras==2.3.1
! pip install termcolor==1.1.0
! pip install six==1.16.0
! pip install tensorflow==1.13.1
! pip install numpy==1.16.2
! pip install matplotlib==2.2.4
! pip install scikit-learn==0.24.2
! pip install h5py==2.10.0

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
import os
import json
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Dense
from keras.layers import GRU, LSTM, Bidirectional
from keras.layers import Embedding, TimeDistributed, Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras_contrib.layers.crf import CRF

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from operator import itemgetter

Using TensorFlow backend.


## Preprocessing

In [3]:
from IPython.display import Image
# get the image
Image(url="JSON-format.png")

In [4]:
py_file_location = ""
sys.path.append(os.path.abspath(py_file_location))

In [5]:
from utils.load_data import get_train_test_pd
from utils.extract_feature import BertVector
from utils.att import Attention
from utils.crf import CRF

# Train

## Encoding Proces

In [6]:
train_df, test_df = get_train_test_pd()
bert_model = BertVector(pooling_strategy="NONE", max_seq_len=512)
print('begin encoding')
f = lambda text: bert_model.encode([text])["encodes"][0]

train_df['x'] = train_df['text'].apply(f)
test_df['x'] = test_df['text'].apply(f)
print('end encoding')

I:PROCESS:[graph:opt: 49]:model config: /Users/slametriyanto/Documents/MyApp/Dissertation_Project/BILSTM_CRF_RE/multi_cased_L-12_H-768_A-12/bert_config.json
I:PROCESS:[graph:opt: 55]:build graph...



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



I:PROCESS:[graph:opt:132]:load parameters from checkpoint...
I:PROCESS:[graph:opt:134]:freeze...
I:PROCESS:[graph:opt:137]:optimize...
I:PROCESS:[graph:opt:148]:write graph to a tmp file: tmp_graph21


begin encoding


KeyboardInterrupt: 

## Training dan Set

In [4]:
x_train = np.array([vec for vec in train_df['x']])
x_test = np.array([vec for vec in test_df['x']])
y_train = np.array([vec for vec in train_df['label']])
y_test = np.array([vec for vec in test_df['label']])

print('x_train: ', x_train.shape)
print('y_train: ', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test: ', y_test.shape)

NameError: name 'train_df' is not defined

In [8]:
num_classes = 4 # {cause of disease, treatment of disease, association, negative}

In [9]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [10]:
print("Size of training input data : ", x_train.shape)
print("Size of training output data : ", y_train.shape)
print("Size of testing input data : ", x_test.shape)
print("Size of testing output data : ", y_test.shape)

Size of training input data :  (1046, 512, 768)
Size of training output data :  (1046, 4)
Size of testing input data :  (261, 512, 768)
Size of testing output data :  (261, 4)


## Model Config

In [29]:
inputs = Input(name="Input", shape=(512, 768))
bilstm = Bidirectional(LSTM(128, dropout=0.2, return_sequences=True, name="Long-Short-Term-Memory"), name="Bidirectional")(inputs)
crf = CRF(num_classes, name="Conditional-Random-Field")(bilstm)
dense = Dense(num_classes, activation='softmax', name="Output")(crf)

model = Model(inputs, dense, name="RE_BiLSTM-CRF")

In [30]:
# If there are .hdf5 files in the original models folder, delete them all
model_dir = 'output'
if os.listdir(model_dir):
    for file in os.listdir(model_dir):
        os.remove(os.path.join(model_dir, file))

## Visualisasi Model

In [31]:
#plot_model(model, to_file='output/model.png', show_shapes=True)

## Proses Pemodelan

In [32]:
model.summary()

Model: "RE_BiLSTM-CRF"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 512, 768)          0         
_________________________________________________________________
Bidirectional (Bidirectional (None, 512, 256)          918528    
_________________________________________________________________
Conditional-Random-Field (CR (None, 256)               1540      
_________________________________________________________________
Output (Dense)               (None, 4)                 1028      
Total params: 921,096
Trainable params: 921,096
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [34]:
# Save the latest val_acc best model file
filepath="output/results/uji2/per-rel-{epoch:02d}-{val_accuracy:.4f}-{batch_size:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [36]:
history = model.fit(
    x_train, y_train, 
    validation_data=(x_test, y_test), 
    batch_size=8, 
    epochs=100, 
    callbacks=[checkpoint]) 
    #callbacks=[early_stopping, checkpoint])

Train on 1046 samples, validate on 261 samples
Epoch 1/100
1046/1046 [==============================] - 41s 39ms/step - loss: 0.9888 - accuracy: 0.5430 - val_loss: 0.9799 - val_accuracy: 0.5862


KeyError: 'batch_size'

In [ ]:
print('The effect on the test set：', model.evaluate(x_test, y_test))

In [ ]:
with open('input/rel_dict.json', 'r', encoding='utf-8') as f:
    label_id_dict = json.loads(f.read())

sorted_label_id_dict = sorted(label_id_dict.items(), key=itemgetter(1))
values = [_[0] for _ in sorted_label_id_dict]

## Model 1

In [ ]:
model = load_model(
  "results/uji1/per-rel-01-0.5287.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 2

In [ ]:
model = load_model(
  "results/uji1/per-rel-01-0.5287.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=64)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 3

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-03-0.6475.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 4

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-05-0.6590.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 5

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-07-0.7356.hdf5", 
  custom_objects={"CRF":CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 6

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-08-0.7395.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 7

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-08-0.7395.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 8

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-08-0.7395.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 9

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-08-0.7395.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))

## Model 10

In [ ]:
model = load_model(
  "/content/drive/MyDrive/Rearch_Dimas/BILSTM_CRF_RE/results/uji 13/per-rel-08-0.7395.hdf5", 
  custom_objects={"CRF": CRF})

y_pred = model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=values))